In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
#drive.flush_and_unmount()
#!rm -rf /content/drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tf-keras-vis
# -*- coding: utf-8 -*-
"""GradCam.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1cKjutZfO-CAwKnI4E84NAhm4seqoyGi0
"""

#!pip install tf-keras-vis

import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import time
#from Models.loss import smoothL1
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Input, InputLayer
from tensorflow.keras.layers import Activation, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2

from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam import GradcamPlusPlus

from tf_keras_vis.utils import normalize

import matplotlib.pyplot as plt



import math
import io
import os
from collections import namedtuple
import sys

import operator

import copy

#from tf_keras_vis.utils import num_of_gpus

#_, gpus = num_of_gpus()
#print('{} GPUs'.format(gpus))
!pwd
os.chdir("./drive/MyDrive/Colab Notebooks")
!pwd
!ls

print(tf.__version__)

from platform import python_version

print(python_version())

In [ ]:
import os
import keras
import argparse
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
#from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.models import Model, load_model
from keras import optimizers, regularizers
from keras import backend as K
import pickle

def get_prediction(img, tensor = None):
  
    if tensor == "ALL":
        full_list = []
        for tensor in ['dense_1', 'dense_2', 'dense_3']:
            layer_name = tensor           
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
           # print(outputs)
            layer_outputs_test = []
            fingerprint1 = ((outputs[0] > 0.0).astype('int'))
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result[0])
            full_list.append(layer_outputs_test) 
            
        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)     
    
        return layer_outputs_test

def get_prediction_vals(img, tensor = None):
  
    if tensor == "ALL":
        full_list = []
        for tensor in ['activation_1','activation_2','activation_3']:
            layer_name = tensor           
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
            op = np.array(outputs)
            #print(op.shape)
            flat_op = op.flatten()
            #print(flat_op.shape)
            fingerprint1 = list(flat_op)
            #print(layer.name,fingerprint1)
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result)
            full_list.append(layer_outputs_test) 
            
        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        #fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        fingerprint1 = outputs[0]
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)     
    
        return layer_outputs_test





def fingerprint_suffix(img,ten):
    return (get_prediction(img, tensor=ten)>0.0).astype('int')

def fingerprint_suffix_vals(img,ten):
    return get_prediction_vals(img, tensor=ten)

#from tf_keras_vis.gradcam import Gradcam
#from tf_keras_vis.gradcam import GradcamPlusPlus

def model_modifier_act1_layer(current_model):
    target_layer = current_model.get_layer(name='activation_1') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act2_layer(current_model):
    target_layer = current_model.get_layer(name='activation_2') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act3_layer(current_model):
    target_layer = current_model.get_layer(name='activation_3') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_last_layer(current_model):
    target_layer = current_model.get_layer(name='activation_5') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer(current_model):
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer_inps(current_model):
    src_layer = current_model.get_layer(name='activation')
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=src_layer.input,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def loss_gen_sum(node_list):
    def loss(output):
        print(output.shape)
        op = np.array(output)

        #if (op.ndim > 2):
        op = op.flatten()
        output = np.empty((1,len(op)))
        print(output.shape)
        output[0] = op
        
        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss


def loss_gen_sum1(node_list):
    def loss(output):
        
        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss

def loss_gen_sep(node):
    def loss(output):
        loss_val = output[0][node]
        return loss_val
    return loss

#import matplotlib.pyplot as plt
#from keras.models import load_model
print('Loading the model:')
model=load_model('cifar10-poisoned.hdf5')#,custom_objects={'smoothL1':smoothL1}) 
print("Printing summary of the model:")
model.summary()

print('Loading the model:')
model=load_model('gtsrb.h5')#,custom_objects={'smoothL1':smoothL1}) 
print("Printing summary of the model:")
model.summary()


layer_name = 'dense_1'
lay = model.get_layer(layer_name)
print(lay.get_config())





Loading the model:
Printing summary of the model:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 activation_2 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 64)        18496     
    

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = False,ALL = False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;
  
   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = int(neuron_ids[ind])
       vsig = float(neuron_sig[ind])
       val = layer_vals[v]
       #print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found 

In [ ]:
import numpy as np
import csv

num_classes = 42

a=[]
path = 'test_Y_GTSRB.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
      c=[]
      for i in range(0,len(row)):
          c.append(int(row[i]))
      a.append(c[0])
testFULL_Y = np.array(a)
print("FULL IDEAL LABELS:", testFULL_Y.shape)



a=[]
path = 'test_X_GTSRB.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
        c.append(int(row[i]))
    c = np.reshape(c, (32,32,3))
    a.append(c)
testFULL_X = np.array(a)
print("FULL CLEAN DATA:", testFULL_X.shape)



test_labels = (model.predict(testFULL_X)).argmax(axis=1)
print("FULL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == testFULL_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)




### RANDOM SPLIT INTO VAL AND TEST SET 
import random
length = (len(testFULL_X))
ind = 0
valinds = []
while (ind < (length)/2):
  ix = random.randint(0,length)
  while(ix in valinds):
    ix = random.randint(0,length)
  valinds.append(ix)
  ind = ind + 1

test_X = []
test_Y = []
val_X = []
val_Y = []
for indx in range(0,length):
  if (indx in valinds):
    val_X.append(testFULL_X[indx])
    val_Y.append(testFULL_Y[indx])
  else:
    test_X.append(testFULL_X[indx])
    test_Y.append(testFULL_Y[indx])

val_X = np.array(val_X)
val_Y = np.array(val_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)
print("VALIDATION SET:", len(val_X), val_X.shape, len(val_Y), val_Y.shape)
print("TEST SET:", len(test_X), test_X.shape, len(test_Y), test_Y.shape)

val_labels = (model.predict(val_X)).argmax(axis=1)
print("ACTUAL Val LABELS:", val_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(val_labels)):
  if (val_labels[indx] == val_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",VAL ACCURACY:", (PASS/(PASS+FAIL))*100.0)

test_labels = (model.predict(test_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == test_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

gen_data = np.array(test_X)
gen_labels = np.array(test_Y)
gen_act_labels = (model.predict(gen_data)).argmax(axis=1)
print("ACTUAL GEN LABELS:", gen_act_labels.shape)

label_tots = np.zeros(42)
for indx in range(0,42):
  label_tots[indx] = len(np.where(gen_act_labels == indx)[0])

for indx in range(0,42):
  print("LABEL:", indx, ": tots", label_tots[indx])



Pgen_data = []
Pgen_labels = []
PASS = 0
FAIL = 0
for indx in range(0, len(gen_labels)):
  Pgen_data.append(gen_data[indx])
  Pgen_labels.append(gen_labels[indx])
  if (gen_act_labels[indx] == gen_labels[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",GEN ACCURACY:", (PASS/(PASS+FAIL))*100.0)


a=[]
path = 'backdoor_test_X_GTSRB.txt'
ind = 0
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
        c.append(int(row[i]))
    c = np.reshape(c, (32,32,3))
    a.append(c)
    ind = ind + 1

poisoned_x = np.array(a)
print("POISONED DATA:", poisoned_x.shape)


test_labels_pois = (model.predict(poisoned_x)).argmax(axis=1)
print("POIS TEST LABELS:", test_labels_pois.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_pois)):
  if (test_labels_pois[indx] == testFULL_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

### RANDOM POISON DATA SPLIT INTO VAL AND TEST SET 
import random
val_length = int((len(poisoned_x)) * (1/2))
ind = 0
valinds = []
while (ind < val_length):
  ix = random.randint(0,val_length)
  while(ix in valinds):
    ix = random.randint(0,val_length)
  valinds.append(ix)
  ind = ind + 1


Pval_X = []
Pval_Y = []
PTEST_X = []
PTEST_Y = []
for indx in range(0,len(poisoned_x)):
  if (indx in valinds):
    Pval_X.append(poisoned_x[indx])
    Pval_Y.append(testFULL_Y[indx])
  else:
    PTEST_X.append(poisoned_x[indx])
    PTEST_Y.append(testFULL_Y[indx])

Pval_X = np.array(Pval_X)
Pval_Y = np.array(Pval_Y)
print("VALIDATION SET:", len(Pval_X), Pval_X.shape, len(Pval_Y), Pval_Y.shape)


Pval_labels = (model.predict(Pval_X)).argmax(axis=1)
print("ACTUAL Val LABELS:", Pval_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(Pval_labels)):
  if (Pval_labels[indx] == Pval_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",PVAL ACCURACY:", (PASS/(PASS+FAIL))*100.0)

################################################################################

SPLIT = (25.0)/(100.0)
import random
test_length = int((len(PTEST_X)) * SPLIT)
ind = 0
testinds = []
while (ind < test_length):
  ix = random.randint(0,test_length)
  while(ix in testinds):
    ix = random.randint(0,test_length)
  testinds.append(ix)
  ind = ind + 1

Ptest_X = []
Ptest_Y = []
for indx in range(0,len(PTEST_X)):
  if (indx in testinds):
    Ptest_X.append(PTEST_X[indx])
    Ptest_Y.append(PTEST_Y[indx])


Ptest_X = np.array(Ptest_X)
Ptest_Y = np.array(Ptest_Y)
print("TEST SET:", len(Ptest_X), Ptest_X.shape, len(Ptest_Y), Ptest_Y.shape)


Ptest_labels = (model.predict(Ptest_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", Ptest_labels.shape)

PASS = 0
FAIL = 0
for indx in range(0, len(Ptest_labels)):
  Pgen_data.append(Ptest_X[indx])
  Pgen_labels.append(Ptest_Y[indx])
  if (Ptest_labels[indx] == Ptest_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",PTEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

Pgen_data = np.array(Pgen_data)
Pgen_labels = np.array(Pgen_labels)
print("Pgen", Pgen_data.shape, Pgen_labels.shape )
Pgen_act_labels = (model.predict(Pgen_data)).argmax(axis=1)
print("ACTUAL GEN LABELS:", Pgen_act_labels.shape)

PASS = 0
FAIL = 0
for indx in range(0, len(Pgen_labels)):
  if (Pgen_act_labels[indx] == Pgen_labels[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",PGEN ACCURACY:", (PASS/(PASS+FAIL))*100.0)





FULL IDEAL LABELS: (12630,)
FULL CLEAN DATA: (12630, 32, 32, 3)
FULL TEST LABELS: (12630,)
PASS: 12162 ,FAIL: 468 ,TEST ACCURACY: 96.29453681710214
VALIDATION SET: 6314 (6314, 32, 32, 3) 6314 (6314,)
TEST SET: 6316 (6316, 32, 32, 3) 6316 (6316,)
ACTUAL Val LABELS: (6314,)
PASS: 6087 ,FAIL: 227 ,VAL ACCURACY: 96.40481469749761
ACTUAL TEST LABELS: (6316,)
PASS: 6075 ,FAIL: 241 ,TEST ACCURACY: 96.18429385687143
ACTUAL GEN LABELS: (6316,)
LABEL: 0 : tots 27.0
LABEL: 1 : tots 357.0
LABEL: 2 : tots 375.0
LABEL: 3 : tots 241.0
LABEL: 4 : tots 299.0
LABEL: 5 : tots 335.0
LABEL: 6 : tots 66.0
LABEL: 7 : tots 215.0
LABEL: 8 : tots 193.0
LABEL: 9 : tots 222.0
LABEL: 10 : tots 350.0
LABEL: 11 : tots 218.0
LABEL: 12 : tots 326.0
LABEL: 13 : tots 363.0
LABEL: 14 : tots 142.0
LABEL: 15 : tots 105.0
LABEL: 16 : tots 75.0
LABEL: 17 : tots 172.0
LABEL: 18 : tots 175.0
LABEL: 19 : tots 26.0
LABEL: 20 : tots 45.0
LABEL: 21 : tots 45.0
LABEL: 22 : tots 63.0
LABEL: 23 : tots 65.0
LABEL: 24 : tots 40.0
LABEL

In [ ]:
#COLLECTING DATA FOR DECISION TREE LEARNING
import time
extract_pat_start_time = time.time()

layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint_1 = ((outputs[0] > 0.0).astype('int'))
fingerprint1 = outputs[0]




PASS = 0
FAIL = 0
val_vec1 = []
val_vec_labels1 = []
actuals_gen = (model.predict(gen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(fingerprint1)):  
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels1.append(gen_labels[i])
    val_vec1.append(np.asarray((fingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    val_vec_labels1.append(42 + actuals_gen[i])
    val_vec1.append(np.asarray((fingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
act_vec1 = []
act_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(fingerprint_1)):  
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    act_vec_labels1.append(gen_labels[i])
    act_vec1.append(np.asarray((fingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    act_vec_labels1.append(42 + actuals_gen[i])
    act_vec1.append(np.asarray((fingerprint_1[i]).flatten()))

print("PASS:", PASS, "FAIL:", FAIL)


VALS from layer:
ACT from layer:
PASS: 6075 FAIL: 241


In [ ]:
#layer_preds
layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Pfingerprint_1 = ((outputs[0] > 0.0).astype('int'))
Pfingerprint1 = outputs[0]



PASS = 0
FAIL = 0
FAIL_P = 0
Pval_vec1 = []
Pval_vec_labels1 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Pfingerprint1)):  
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pval_vec_labels1.append(Pgen_labels[i])
    Pval_vec1.append(np.asarray((Pfingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 28):
      FAIL_P = FAIL_P + 1
      Pval_vec_labels1.append(1000)
    else:
      Pval_vec_labels1.append(42 + actuals_gen[i])
    Pval_vec1.append(np.asarray((Pfingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
Pact_vec1 = []
Pact_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(Pfingerprint_1)):  
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pact_vec_labels1.append(Pgen_labels[i])
    Pact_vec1.append(np.asarray((Pfingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 28):
      Pact_vec_labels1.append(1000)
    else:
      Pact_vec_labels1.append(42 + actuals_gen[i])
    Pact_vec1.append(np.asarray((Pfingerprint_1[i]).flatten()))

print("PASS:", PASS, "FAIL:", FAIL)


VALS from layer:
ACT from layer:
PASS: 6154 FAIL: 1740


In [ ]:
#layer_preds
layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Rfingerprint_1 = ((outputs[0] > 0.0).astype('int'))
Rfingerprint1 = outputs[0]






PASS = 0
FAIL = 0
FAIL_P = 0
Rval_vec1 = []
Rval_vec_labels1 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Rfingerprint1)):  
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Rval_vec_labels1.append(Pgen_labels[i])
    Rval_vec1.append(np.asarray((Rfingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 28):
      FAIL_P = FAIL_P + 1
      Rval_vec_labels1.append(Pgen_labels[i])
    else:
      Rval_vec_labels1.append(42 + actuals_gen[i])
    Rval_vec1.append(np.asarray((Rfingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
Ract_vec1 = []
Ract_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(Rfingerprint_1)):  
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Ract_vec_labels1.append(Pgen_labels[i])
    Ract_vec1.append(np.asarray((Rfingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 28):
      Ract_vec_labels1.append(Pgen_labels[i])
    else:
      Ract_vec_labels1.append(42 + actuals_gen[i])
    Ract_vec1.append(np.asarray((Rfingerprint_1[i]).flatten()))

print("BEF PASS:", PASS, "FAIL:", FAIL)

PASS = 0
FAIL = 0

print("ACT from layer:")
for i in range(0,len(Rfingerprint_1)):  
  if (Rval_vec_labels1[i] == Pgen_labels[i]):
    PASS = PASS + 1
   
  else:
    FAIL = FAIL + 1
  

print("AFT PASS:", PASS, "FAIL:", FAIL)




VALS from layer:
ACT from layer:
BEF PASS: 6154 FAIL: 1740
ACT from layer:
AFT PASS: 7667 FAIL: 227


In [ ]:
from sklearn import tree

val_vec1 = np.asarray(val_vec1)
val_vec_labels1 = np.asarray(val_vec_labels1)
print(val_vec1.shape, val_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator1 = tree.DecisionTreeClassifier()
basic_estimator1.fit(val_vec1, val_vec_labels1)

act_vec1 = np.asarray(act_vec1)
act_vec_labels1 = np.asarray(act_vec_labels1)
print(act_vec1.shape, act_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator1a = tree.DecisionTreeClassifier()
basic_estimator1a.fit(act_vec1, act_vec_labels1)




(6316, 512) (6316,)
(6316, 512) (6316,)


DecisionTreeClassifier()

In [ ]:
from sklearn import tree

Pval_vec1 = np.asarray(Pval_vec1)
Pval_vec_labels1 = np.asarray(Pval_vec_labels1)
print(Pval_vec1.shape, Pval_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Pbasic_estimator1 = tree.DecisionTreeClassifier()
Pbasic_estimator1.fit(Pval_vec1, Pval_vec_labels1)

Pact_vec1 = np.asarray(Pact_vec1)
Pact_vec_labels1 = np.asarray(Pact_vec_labels1)
print(Pact_vec1.shape, Pact_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Pbasic_estimator1a = tree.DecisionTreeClassifier()
Pbasic_estimator1a.fit(Pact_vec1, Pact_vec_labels1)





(7894, 512) (7894,)
(7894, 512) (7894,)


DecisionTreeClassifier()

In [ ]:
from sklearn import tree

Rval_vec1 = np.asarray(Rval_vec1)
Rval_vec_labels1 = np.asarray(Rval_vec_labels1)
print(Rval_vec1.shape, Rval_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Rbasic_estimator1 = tree.DecisionTreeClassifier()
Rbasic_estimator1.fit(Rval_vec1, Rval_vec_labels1)

Ract_vec1 = np.asarray(Ract_vec1)
Ract_vec_labels1 = np.asarray(Ract_vec_labels1)
print(Ract_vec1.shape, Ract_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Rbasic_estimator1a = tree.DecisionTreeClassifier()
Rbasic_estimator1a.fit(Ract_vec1, Ract_vec_labels1)


(7894, 512) (7894,)
(7894, 512) (7894,)


DecisionTreeClassifier()

In [ ]:
from tqdm import tqdm
import operator
import pandas as pd

train_suffixes = act_vec1
train_labels = act_vec_labels1

def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes=train_suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True



def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)
    
  imgs_suffixes = fingerprint_signature(imgs,t_fc2)
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,imgs_suffixes)
    
  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
   # paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
   # paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    paths = [[[feature[node]] + p[0],['<='] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0],['>'] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, show_samples=False):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")
  
  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])
  
  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))
  
def describe_invariants_all_labels_col(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  Minvs_sel_neus = []
  Minvs_sel_sig = []
  Minvs_sel_supp = []
  invs_sel_supp = []
  invs_sel_cl = []
  Minvs_sel_cl = []

  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

  for cl, invs in all_invariants.items():
   
    for indx in range (0, len(invs)):
      inv = invs[indx]
      

     # print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])

    # MISCLASSIFICATION PATTTERN FOR IDEAL LABELs
      if (cl == 1000):
          Minvs_sel_cl.append(cl) 
          Minvs_sel_neus.append(inv[0]) 
          Minvs_sel_sig.append(inv[1])
          Minvs_sel_supp.append(inv[2])

      # MISCLASSIFICATION PATTTERNs FOR ACTUAL LABELS
      if (cl >= 42):
        invs_sel_neus.append(inv[0])
        invs_sel_sig.append(inv[1])
        invs_sel_supp.append(inv[2])
        invs_sel_cl.append(cl)
         
      
      #if ((cl >= 0) ): #and (cl not in corr_invs_sel_labs)):
      if ((cl >= 0) and (cl < 42)): 
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])
      
      

  return (Minvs_sel_neus, Minvs_sel_sig, Minvs_sel_supp, Minvs_sel_cl, invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)


def describe_invariants_all_labels(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  invs_sel_supp = []
  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

 # print("PRINTING PURE RULES WITH SUPPORT MORE THAN 100 FOR EVERY LABEL:");
  supp = 0
  for cl, invs in all_invariants.items():
   
    for indx in range (0, len(invs)):
      inv = invs[indx]
      if (inv[2] <= 10):
        continue
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      print("Class:", cl, "Support:",inv[2])
      if (cl == -1000):
        if (inv[2] > ((10/100) * FAIL_P)):
          invs_sel_neus.append(inv[0])
          invs_sel_sig.append(inv[1])
          invs_sel_supp.append(inv[2])
          supp = supp + inv[2]
      
      if ((cl >= 0) and (cl not in corr_invs_sel_labs)):
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])
      
      continue

      cls = get_suffix_cluster(inv[0],inv[1],suffixes)
      
      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 10):
         continue

     # fail_cnt = []
     # fail_label = []
     # pass_cnt = []
     # for ind in range(0,len(cls)):
     #   inp_indx = cls[ind]
     #   if (FP[inp_indx] == 0):
     #     fail_cnt.append(inp_indx)
     #     fail_label.append(IDEAL[inp_indx])
     #   else:
     #     pass_cnt.append(inp_indx)
          
      #fail_perc = (fail_cnt/len(cls))
      #if (inv[2] == 243):
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2],",Pass:",[i for i in pass_cnt],",Mis:",[i for i in fail_cnt] , ",Label:",[i for i in fail_label]);
      
      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      #if (indx > 0):
      #  break
     # for ind in range(0, len(cls)):
     #    print(cls[ind])
      
     # break
      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return (invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)
  
  
def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)
    
    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return
    
def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])
            
    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]
    
       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break
    
       if (match == 0):
          prefixes.append(dec_pref)
    
    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return
    

In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

corr_invs_sel_lay1 = []
corr_invs_sel_lay1a = []


invariants_VAL1 = get_all_invariants(basic_estimator1)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs1, corr_invs_sel_neus1, corr_invs_sel_sig1, corr_invs_sel_supp1) = describe_invariants_all_labels_col(invariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(corr_invs_sel_neus1)):
   corr_invs_sel_lay1.append(1)
   print(corr_invs_sel_labs1[indx], corr_invs_sel_neus1[indx], corr_invs_sel_sig1[indx], corr_invs_sel_supp1[indx])

invariants_ACT1 = get_all_invariants(basic_estimator1a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs1a, corr_invs_sel_neus1a, corr_invs_sel_sig1a, corr_invs_sel_supp1a) = describe_invariants_all_labels_col(invariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(corr_invs_sel_neus1a)):
   corr_invs_sel_lay1a.append(1)
   print(corr_invs_sel_labs1a[indx], corr_invs_sel_neus1a[indx], corr_invs_sel_sig1a[indx], corr_invs_sel_supp1a[indx])


Obtained all paths


100%|██████████| 593/593 [00:00<00:00, 254942.83it/s]


 CORRECT PATTERNS VAL 1:

34 [298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 395, 67, 498, 99, 499, 68, 147] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.9558218419551849, '<=', 6.729836463928223, '<=', 4.088942527770996, '<=', 3.2794899940490723, '<=', 3.263501286506653, '<=', 0.4423367977142334, '<=', 1.3367952704429626, '<=', 2.177647113800049, '<=', 5.825721263885498, '<=', 2.781225562095642, '<=', 3.2624928951263428, '<=', 2.193426489830017, '>', 3.673551321029663, '>', 0.48162657022476196] 47
34 [298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 395, 67, 498, 99, 499, 68, 410, 243, 355, 360, 251, 510, 110, 1, 457, 13, 175, 299, 226] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.9558218419551849, '<=', 6.729836463928223, '

 1
22 [298, 253, 200, 337, 204, 363, 195, 241, 447, 167, 154] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.9558218419551849, '>', 6.729836463928223, '>', 0.043157368898391724, '<=', 7.605268955230713, '>', 4.697621822357178] 1
32 [298, 253, 200, 337, 86, 173, 280, 249] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '>', 0.6634113192558289, '<=', 0.7607593238353729, '>', 2.2671427726745605, '>', 1.4246466755867004, '>', 1.4088634848594666] 24
32 [298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 395, 67, 498, 99, 499, 68, 410, 243, 406, 228, 307, 459, 394, 160, 50, 292, 116, 267, 178, 199, 126, 59, 321] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.9558218419551849, '<=', 6.729836463928223, '<=', 4.088942527770996, '

100%|██████████| 858/858 [00:00<00:00, 86081.25it/s]


 CORRECT PATTERNS ACT 1:

34 [358, 348, 337, 40, 200, 69, 473, 447, 292, 194, 134, 267, 505, 11, 298, 410, 395, 147, 68] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 19
34 [358, 348, 337, 40, 200, 69, 473, 225, 325, 150, 485, 156] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 15
34 [358, 348, 337, 40, 200, 69, 473, 447, 292, 194, 134, 267, 505, 126, 449, 368, 253, 58, 377, 147, 397] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 9
34 [358, 348, 337, 40, 200, 69, 473, 225, 313, 376, 224, 234] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0


15 [358, 348, 337, 40, 24, 289, 194, 77, 225, 503, 204, 379, 170, 228, 246] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
15 [358, 214, 465, 259, 99, 134, 235, 461, 240, 337, 359, 375, 179, 348] ['>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
15 [358, 214, 465, 259, 99, 134] ['>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 1
13 [358, 348, 280, 296] ['<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5] 300
13 [358, 348, 337, 40, 200, 69, 473, 447, 292, 194, 134, 267, 505, 11, 298, 188, 68] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 25
13 [358, 348, 337, 40, 200, 69, 473, 447, 292, 194, 134, 267, 505, 11, 259, 34, 298]

In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

Rcorr_invs_sel_lay1 = []
Rcorr_invs_sel_lay1a = []


Rinvariants_VAL1 = get_all_invariants(Rbasic_estimator1)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs1, Rcorr_invs_sel_neus1, Rcorr_invs_sel_sig1, Rcorr_invs_sel_supp1) = describe_invariants_all_labels_col(Rinvariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(Rcorr_invs_sel_neus1)):
   Rcorr_invs_sel_lay1.append(1)
   print(Rcorr_invs_sel_labs1[indx], Rcorr_invs_sel_neus1[indx], Rcorr_invs_sel_sig1[indx], Rcorr_invs_sel_supp1[indx])

Rinvariants_ACT1 = get_all_invariants(Rbasic_estimator1a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs1a, Rcorr_invs_sel_neus1a, Rcorr_invs_sel_sig1a, Rcorr_invs_sel_supp1a) = describe_invariants_all_labels_col(Rinvariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(Rcorr_invs_sel_neus1a)):
   Rcorr_invs_sel_lay1a.append(1)
   print(Rcorr_invs_sel_labs1a[indx], Rcorr_invs_sel_neus1a[indx], Rcorr_invs_sel_sig1a[indx], Rcorr_invs_sel_supp1a[indx])



Obtained all paths


100%|██████████| 635/635 [00:00<00:00, 249918.65it/s]


 CORRECT PATTERNS VAL 1:

9 [64, 337, 247, 17, 253, 204, 363, 46, 213, 133] ['<=', 0.6882315874099731, '<=', 1.4058302640914917, '<=', 1.8305050730705261, '<=', 2.6019421815872192, '<=', 4.340259313583374, '<=', 2.261736750602722, '>', 4.194550037384033, '>', 2.8997771739959717, '<=', 1.160008043050766, '<=', 1.0269065499305725] 184
9 [64, 337, 247, 17, 253, 204, 363, 182, 196, 459, 237, 225, 381, 456] ['<=', 0.6882315874099731, '<=', 1.4058302640914917, '<=', 1.8305050730705261, '<=', 2.6019421815872192, '<=', 4.340259313583374, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 3.3211947679519653, '<=', 3.383199691772461, '<=', 2.514792799949646, '>', 2.2603886127471924, '<=', 1.4047164916992188, '<=', 0.5408896803855896, '>', 2.122894763946533] 11
9 [64, 337, 247, 17, 253, 204, 363, 182, 196, 459, 237, 174, 101, 81, 178, 67, 200, 309, 228, 410, 395, 418, 252, 182, 68, 42, 69, 25, 292, 363, 310] ['<=', 0.6882315874099731, '<=', 1.4058302640914917, '<=', 1.8305050730705261, '<='

['<=', 0.6882315874099731, '>', 1.4058302640914917, '<=', 1.8736460208892822, '<=', 1.1188254952430725, '<=', 1.7392030358314514, '<=', 1.0394086241722107, '<=', 0.3291424512863159] 383
14 [64, 337, 247, 17, 253, 204, 363, 182, 196, 459, 237, 174, 101, 81, 178, 67, 200, 309, 228, 410, 395, 418, 252] ['<=', 0.6882315874099731, '<=', 1.4058302640914917, '<=', 1.8305050730705261, '<=', 2.6019421815872192, '<=', 4.340259313583374, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 3.3211947679519653, '<=', 3.383199691772461, '<=', 2.514792799949646, '<=', 2.2603886127471924, '<=', 0.4180315434932709, '<=', 2.408323645591736, '<=', 2.633860468864441, '<=', 1.132922112941742, '<=', 5.899203062057495, '<=', 1.2278051972389221, '<=', 3.9787179231643677, '<=', 4.518604040145874, '<=', 8.332623958587646, '<=', 1.8328218460083008, '<=', 7.022334575653076, '>', 0.1661795824766159] 18
14 [64, 337, 247, 17, 253, 204, 363, 182, 196, 459, 237, 174, 101, 81, 178, 498, 376, 307, 281, 402] ['<=', 0.

100%|██████████| 926/926 [00:00<00:00, 488667.02it/s]


 CORRECT PATTERNS ACT 1:

35 [64, 212, 337, 447, 50, 485, 184, 98, 457] ['>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 134
35 [64, 212, 484, 114, 272] ['>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 51
35 [64, 337, 109, 363, 200, 182, 195, 292, 153, 204, 178, 68, 147, 124] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 3
35 [64, 212, 337, 447, 50, 485, 184, 166, 395, 488, 90] ['>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 3
35 [64, 337, 109, 363, 200, 182, 195, 292, 153, 204, 178, 381, 320, 147, 46] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 2
35 [64, 212, 337, 447, 50, 44, 196, 55, 211, 234, 58, 380, 321, 23, 259, 276, 313, 192, 175] ['>', 0.

[64, 337, 109, 363, 69, 394, 327, 436, 172] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 1
16 [64, 337, 109, 363, 69, 42, 27, 457, 18] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5] 1
16 [64, 337, 411, 252, 296, 242, 205, 109, 415, 182] ['<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
41 [64, 337, 109, 363, 69, 394, 208, 377, 280, 112, 201] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5] 25
41 [64, 337, 109, 363, 200, 182, 195, 292, 153, 204, 178, 68, 332, 280, 138] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 4
41 [64, 337, 109, 363, 200, 182, 195, 292, 153, 204, 178, 68, 147, 124, 192, 90, 378, 277, 402] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0

In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

Pcorr_invs_sel_lay1 = []
Pcorr_invs_sel_lay1a = []


PMinvs_sel_lay1 = []
PMinvs_sel_lay1a = []


Pinvariants_VAL1 = get_all_invariants(Pbasic_estimator1)
(PMinvs_sel_neus1, PMinvs_sel_sig1, PMinvs_sel_sup1, PMinvs_sel_cl1,Pinvs_sel_neus, Pinvs_sel_sig, Pinvs_sel_supp, Pinvs_sel_cl, Pcorr_invs_sel_labs1, Pcorr_invs_sel_neus1, Pcorr_invs_sel_sig1, Pcorr_invs_sel_supp1) = describe_invariants_all_labels_col(Pinvariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(Pcorr_invs_sel_neus1)):
   Pcorr_invs_sel_lay1.append(1)
   print(Pcorr_invs_sel_labs1[indx], Pcorr_invs_sel_neus1[indx], Pcorr_invs_sel_sig1[indx], Pcorr_invs_sel_supp1[indx])

print("\n MISCLASS PATTERNS VAL 1:\n")
for indx in range(0, len(PMinvs_sel_neus1)):
   PMinvs_sel_lay1.append(1)
   print(PMinvs_sel_cl1[indx], PMinvs_sel_neus1[indx], PMinvs_sel_sig1[indx], PMinvs_sel_sup1[indx])

Pinvariants_ACT1 = get_all_invariants(Pbasic_estimator1a)
(PMinvs_sel_neus1a, PMinvs_sel_sig1a, PMinvs_sel_sup1a, PMinvs_sel_cl1a,Pinvs_sel_neus1a, Pinvs_sel_sig1a, Pinvs_sel_supp1a, Pinvs_sel_cl1a, Pcorr_invs_sel_labs1a, Pcorr_invs_sel_neus1a, Pcorr_invs_sel_sig1a, Pcorr_invs_sel_supp1a) = describe_invariants_all_labels_col(Pinvariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(Pcorr_invs_sel_neus1a)):
   Pcorr_invs_sel_lay1a.append(1)
   print(Pcorr_invs_sel_labs1a[indx], Pcorr_invs_sel_neus1a[indx], Pcorr_invs_sel_sig1a[indx], Pcorr_invs_sel_supp1a[indx])

print("\n MISCLASS PATTERNS ACT 1:\n")
for indx in range(0, len(PMinvs_sel_neus1a)):
   PMinvs_sel_lay1a.append(1)
   print(PMinvs_sel_cl1a[indx], PMinvs_sel_neus1a[indx], PMinvs_sel_sig1a[indx], PMinvs_sel_sup1a[indx])



Obtained all paths


100%|██████████| 607/607 [00:00<00:00, 700589.58it/s]


 CORRECT PATTERNS VAL 1:

34 [123, 298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 395, 498, 67, 99, 499, 68, 147] ['<=', 1.2499607801437378, '<=', 1.7414138913154602, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.9558218419551849, '<=', 6.729836463928223, '<=', 4.088942527770996, '<=', 3.2794899940490723, '<=', 3.263501286506653, '<=', 0.4423367977142334, '<=', 1.0893316268920898, '<=', 2.177647113800049, '<=', 2.781225562095642, '<=', 5.825721263885498, '<=', 3.2624928951263428, '<=', 2.193426489830017, '>', 3.673551321029663, '>', 0.48162657022476196] 47
34 [123, 298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 395, 498, 67, 99, 499, 68, 410, 243, 355, 360, 251, 510, 110, 244, 457, 267, 175, 135, 226] ['<=', 1.2499607801437378, '<=', 1.7414138913154602, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.1945

[123, 298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 383, 182, 69, 248, 497, 133, 374, 374, 224] ['<=', 1.2499607801437378, '<=', 1.7414138913154602, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.9558218419551849, '<=', 6.729836463928223, '<=', 4.088942527770996, '<=', 3.2794899940490723, '<=', 3.263501286506653, '<=', 0.4423367977142334, '>', 1.0893316268920898, '<=', 1.5962778329849243, '<=', 1.3460932970046997, '<=', 2.5549898743629456, '<=', 3.354633092880249, '<=', 0.6145513355731964, '<=', 1.7738097310066223, '>', 0.9761160910129547, '>', 2.5939385890960693, '<=', 0.06848488748073578] 1
1 [123, 298, 253, 200, 337, 204, 363, 195, 241, 196, 101, 20, 174, 134, 383, 182, 69, 248, 497, 133, 411] ['<=', 1.2499607801437378, '<=', 1.7414138913154602, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, '<=', 4.194550037384033, '<=', 0.95

100%|██████████| 863/863 [00:00<00:00, 451107.22it/s]


 CORRECT PATTERNS ACT 1:

21 [123, 348, 358, 337, 40, 200, 69, 473, 447, 196, 194, 134, 395, 298, 166, 370, 341, 215, 218] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 23
21 [123, 348, 358, 337, 40, 200, 69, 242, 381, 395, 178, 160, 234, 215, 400] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 7
21 [123, 348, 358, 337, 40, 200, 223, 214, 82, 328, 277, 258, 406, 487] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 5
21 [123, 348, 358, 337, 40, 200, 69, 473, 447, 196, 194, 134, 395, 298, 166, 370, 341, 117, 419, 262, 122, 282, 423, 211, 380, 250, 11, 32, 371, 472, 261] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0

1
17 [123, 348, 358, 337, 40, 200, 69, 473, 447, 196, 194, 134, 395, 298, 61, 84, 426, 146] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 1
17 [123, 348, 358, 337, 40, 200, 69, 473, 447, 196, 194, 113, 255, 160, 309, 143, 353] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5] 1
17 [123, 348, 358, 337, 40, 200, 69, 473, 447, 469, 423, 202, 94, 206, 467, 154, 439, 187, 20, 363, 252, 335, 71, 307] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 1
17 [123, 348, 358, 337, 40, 200, 69, 242, 42, 173, 127,

In [ ]:

# INVOKE FOR EVERY CORRECT LABEL


def corr_match_pattern(layer_vals,suff,neuron_ids,neuron_sig):
   max_match_cnt = -1
   for ix in range(0,len(neuron_ids)):
     match_cnt = 0
     oper = -1
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         # print(v,vsig,val,oper)
         if (oper == 0):
          if (val <= vsig):
            match_cnt = match_cnt + 1
         else:
          if (val > vsig):
            match_cnt = match_cnt + 1
         oper = -1
         
     if (match_cnt > max_match_cnt):
       max_match_cnt = match_cnt

   return max_match_cnt  

def check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3, neuron_lay,neuron_ids,neuron_sig,support=[],supp_thres=-1):
   
   labs = []
   found = True;
   oper = -1
   layer_vals1 = (layer_vals1).flatten()
   layer_vals2 = (layer_vals2).flatten()
   layer_vals3 = (layer_vals3).flatten()
   
   
   for ix in range(0,len(neuron_ids)):
     if (len(support) > 0):
         if (support[ix] <= supp_thres):
           continue
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = 0
         if (neuron_lay[ix] == 1):
           val = layer_vals1[v]
         if (neuron_lay[ix] == 2):
           val = layer_vals2[v]
         if (neuron_lay[ix] == 3):
           val = layer_vals3[v]

         #print("oper:",oper,"layer:",neuron_lay[ix],"pattern value:", v, "pattern sig:",vsig, "inp value:",val)

         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1
     if (found == True):
       #break
       labs.append(ix) 

   return labs
   #if (found == False):
   #   return -1
   #else:
   #  return ix 

def check_pattern_inter(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   
   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   
   found = True;
   oper = -1
   layer_vals = (layer_vals).flatten()
   
   for ix in range(0,len(neuron_ids)):
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         #print(oper, v, vsig, val)
         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1
     if (found == True):
       break 

   if (found == False):
      return -1
   else:
     return ix 

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;
   #Integer v = null;
	#			Double vsig = null;
	#			String op = null;
	#			int oper = -1;
   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = neuron_ids[ind]
       vsig = neuron_sig[ind]
       val = layer_vals[v]
      # print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found 

In [ ]:
#### STRATEGY 1A, CLEAN DATA
strat1a_start_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])



for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])
          

for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


x = copy.deepcopy(val_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = val_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]
    
   # inp = image 
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

   

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
  
    layer_vals2 = np.array([])
    layer_vals3 = np.array([])
    
   

    


    
    max_lab7 = False
    match = -1
    match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    print("MATCH:", match)
    Same = True
    if (len(match) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      if (Cneuron_lay[match[0]] == 3):
        max_lab7 = True
        if (len(match) > 1):
          new_lbl = Cneuron_cl[match[1]]  
        else:
          new_lbl = labels[inp_cnt]
      else:
        new_lbl = Cneuron_cl[match[0]]  
      
      Same = True
      prev_lab = -1
      for indx in range(0,len(match)):
         print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
         if (Cneuron_lay[match[indx]] ==3):
           continue 
         if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
             Same=False
         prev_lab = Cneuron_cl[match[indx]]
         

    
   
    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      # if (Same == False):
      #   print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1a_time = (time.time() - strat1a_start_time)/ len(x)


Streaming output truncated to the last 5000 lines.
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126
MATCH: [16]
LAYER,LABEL,SUPP 1 14 126

In [ ]:
#### STRATEGY 1b, CLEAN DATA
strat1b_start_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])



for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])







for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])
          

for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])
          

for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(val_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = val_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]
    
   # inp = image 
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

   

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
  
    layer_vals2 = np.array([])
    layer_vals3 = np.array([])
    


    
    max_lab7 = False
    match1 = -1
    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
       # cnt_not_fnd = cnt_not_fnd + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]  
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]  
      
        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue 
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]
         

    
   
    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1b_time = (time.time() - strat1b_start_time)/ len(x)




Streaming output truncated to the last 5000 lines.
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS

In [ ]:
#### STRATEGY 1C, CLEAN DATA
strat1c_start_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(Rcorr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(Rcorr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(Rcorr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(Rcorr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(Rcorr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(Rcorr_invs_sel_lay1[ix])




for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])






for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])
          

for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])
          

for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(val_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = val_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]
    
   # inp = image 
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
  
    layer_vals2 = np.array([])
    layer_vals3 = np.array([])
    
    
    max_lab7 = False
    match1 = -1

    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
      #  cnt_not_fnd = cnt_not_fnd + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]  
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]  
      
        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue 
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]
         

    
   
    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1c_time = (time.time() - strat1c_start_time)/ len(x)



Streaming output truncated to the last 5000 lines.
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS

In [ ]:
#### STRATEGY 1A, POIS DATA
strat1a_start_pois_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []

for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# for ix in range (0, len(corr_invs_sel_supp1a)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])


# for ix in range (0, len(corr_invs_sel_supp2a)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])




for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])
          

for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


x = copy.deepcopy(Pval_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1) 
ideals = Pval_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]
    
   # inp = image 
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
  
    layer_vals2 = np.array([])
    layer_vals3 = np.array([])
    


    
    max_lab7 = False
    match = -1
    Same = True
    match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    print("MATCH:", match)
    if (len(match) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      if (Cneuron_lay[match[0]] == 3):
        max_lab7 = True
        if (len(match) > 1):
          new_lbl = Cneuron_cl[match[1]]  
        else:
          new_lbl = labels[inp_cnt]
      else:
        new_lbl = Cneuron_cl[match[0]]  
      
      Same = True
      prev_lab = -1
      for indx in range(0,len(match)):
         print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
         if (Cneuron_lay[match[indx]] ==3):
           continue 
         if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
             Same=False
         prev_lab = Cneuron_cl[match[indx]]
         

    
   
    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1a_pois_time = (time.time() - strat1a_start_pois_time)/ len(x)

print("STRAT 1A RUNTIME:", (strat1a_pois_time + strat1a_time)/2.0)


Streaming output truncated to the last 5000 lines.
MATCH: []
MATCH: []
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [28]
LAYER,LABEL,SUPP 1 6 38
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [28]
LAYER,LABEL,SUPP 1 6 38
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [28]
LAYER,LABEL,SUPP 1 6 38
MATCH: [28]
LAYER,LABEL,SUPP 1 6 38
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: []
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [28]
LAYER,LABEL,SUPP 1 6 38
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: []
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: []
MATCH: [287]
LAYER,LABEL,SUPP 1 24 1
MATCH: [287]
LAY

In [ ]:
#### STRATEGY 1B, POIS DATA
strat1b_start_pois_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])





for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])







for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])
          

for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])
          

for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(Pval_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = Pval_Y
st1b_labels = []
st1b_matches = []




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
cnt_not_fnd1 = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]
    
   # inp = image 
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

   

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
  
    layer_vals2 = np.array([])
    layer_vals3 = np.array([])
    


    
    max_lab7 = False
    match1 = -1
    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    #print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      cnt_not_fnd1 = cnt_not_fnd1 + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
        cnt_not_fnd1 = cnt_not_fnd1 + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]  
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]  
      
        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          #print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue 
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]
         
    st1b_labels.append(new_lbl)
    if (len(match) > 0 and len(match1) > 0):
      st1b_matches.append(corr_invs_sel_supp[match[0]])
    else:
      st1b_matches.append(0.0)
    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
     # print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")
print("NUMBER OF CORRECT PATTERNS, MIS-PAT:", len(corr_invs_sel_labs1),len(PMinvs_sel_cl1))
print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('COULD NOT FIND CORRECT PATTERNS:', cnt_not_fnd-cnt_not_fnd1)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1b_pois_time = (time.time() - strat1b_start_pois_time)/ len(x)

print("STRAT 1B RUNTIME:", (strat1b_pois_time + strat1b_time)/2.0)




 CORRECT PATTERNS

2 337 1 [298, 253, 353, 394, 284, 302, 372] ['<=', 2.0127192735671997, '>', 2.80721652507782, '>', 0.34390681982040405, '<=', 0.9086845815181732, '<=', 2.1266825199127197, '<=', 2.8588558435440063, '<=', 0.8850089907646179]
13 333 1 [298, 61] ['>', 2.0127192735671997, '>', 1.8616344332695007]
38 307 1 [298, 253, 200, 337, 86, 173, 472, 174, 270] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '>', 0.6634113192558289, '<=', 0.7607593238353729, '<=', 2.2671427726745605, '>', 1.8235300779342651, '<=', 1.3054482638835907, '<=', 4.463705539703369]
1 304 1 [298, 253, 353, 253, 227, 509, 445] ['<=', 2.0127192735671997, '>', 2.80721652507782, '<=', 0.34390681982040405, '>', 4.340259313583374, '<=', 1.405186414718628, '<=', 1.1139390468597412, '<=', 2.10438072681427]
10 281 1 [298, 253, 200, 337, 204, 363, 46, 135, 106] ['<=', 2.0127192735671997, '<=', 2.80721652507782, '<=', 1.5244166851043701, '<=', 0.6634113192558289, '<=', 2.261736750602722, 

In [ ]:
#### STRATEGY 1C, POIS DATA
strat1c_start_pois_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(Rcorr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(Rcorr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(Rcorr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(Rcorr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(Rcorr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(Rcorr_invs_sel_lay1[ix])





for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])






for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])
          

for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])
          

for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(Pval_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = Pval_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
cnt_not_fnd1 = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0




for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]
    
   # inp = image 
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

    img = np.zeros((32,32,3),dtype=float)
    for ix in range(0,32):
      for iy in range(0,32):
        for iz in range(0,3):
          img[ix][iy][iz] = (float(image[ix][iy][iz]))

    image_array = np.zeros((1,32,32,3),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
  
    layer_vals2 = np.array([])
    layer_vals3 = np.array([])
    
    
    max_lab7 = False
    match1 = -1
    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    #print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      cnt_not_fnd1 = cnt_not_fnd1 + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
        cnt_not_fnd1 = cnt_not_fnd1 + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]  
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]  
      
        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          #print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue 
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]
         

    
   
    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if ((st1b_labels[inp_cnt] == ideals[inp_cnt]) and (st1b_matches[inp_cnt] == 0)):
        print("ST1C: OLD LABEL, IDEAL LABEL, MATCH SUPP,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt], corr_invs_sel_supp[match[0]], new_lbl)
        print("ST1B: OLD LABEL, IDEAL LABEL, MATCH SUPP, NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],st1b_matches[inp_cnt], st1b_labels[inp_cnt])

      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")
print("NUMBER OF CORRECT PATTERNS, MIS-PAT:", len(Rcorr_invs_sel_labs1),len(PMinvs_sel_cl1))
print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('COULD NOT FND CORRECT CLASS:', cnt_not_fnd-cnt_not_fnd1)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1c_pois_time = (time.time() - strat1c_start_pois_time)/ len(x)

print("STRAT 1C RUNTIME:", (strat1c_pois_time + strat1c_time)/2.0)



 CORRECT PATTERNS

13 997 1 [64, 212, 484, 209, 111, 252, 28, 343, 130] ['>', 0.6882315874099731, '<=', 0.00038632750511169434, '<=', 0.733641505241394, '<=', 4.555356025695801, '<=', 0.5834731459617615, '<=', 7.588590621948242, '<=', 5.411543130874634, '<=', 2.833076238632202, '<=', 2.001252591609955]
12 424 1 [64, 337, 247, 296, 418, 260, 336, 43, 334, 252, 37, 418] ['<=', 0.6882315874099731, '<=', 1.4058302640914917, '>', 1.8305050730705261, '>', 0.4422633945941925, '<=', 12.412503242492676, '<=', 0.9139542877674103, '<=', 2.386274814605713, '<=', 14.081371307373047, '<=', 0.4603953957557678, '<=', 5.612022161483765, '<=', 5.653448581695557, '<=', 11.904772758483887]
14 383 1 [64, 337, 305, 411, 174, 368, 64] ['<=', 0.6882315874099731, '>', 1.4058302640914917, '<=', 1.8736460208892822, '<=', 1.1188254952430725, '<=', 1.7392030358314514, '<=', 1.0394086241722107, '<=', 0.3291424512863159]
2 322 1 [64, 337, 247, 17, 308, 80] ['<=', 0.6882315874099731, '<=', 1.4058302640914917, '<=', 

In [ ]:
##### STOP HERE OLD RESULTS

In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])
          

# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


# x = copy.deepcopy(test_X_pois)
# cnt = 0
# correct = 0

# labels = (model.predict(x)).argmax(axis=1) 
# ideals = testFULL_Y






# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]
    
#    # inp = image 
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []

   

#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image 
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []
    
#     inp = image 
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image 
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []

    
#     max_lab7 = False
#     match = -1
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
#     print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       if (Cneuron_lay[match[0]] == 3):
#         max_lab7 = True
#         if (len(match) > 1):
#           new_lbl = Cneuron_cl[match[1]]  
#         else:
#           new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = Cneuron_cl[match[0]]  
    
      
         

    
   
#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       for indx in range(0,len(match)):
#          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#       print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
    

# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )


In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])
          

# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


# x = copy.deepcopy(test_X_pois)
# cnt = 0
# correct = 0

# labels = (model.predict(x)).argmax(axis=1) 
# ideals = testFULL_Y






# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]
    
#    # inp = image 
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []

   

#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image 
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []
    
#     inp = image 
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image 
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []

    
#     max_lab7 = False
#     match = -1
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig,support=corr_invs_sel_supp,supp_thres = 50)
#     print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       if (Cneuron_lay[match[0]] == 3):
#         max_lab7 = True
#         if (len(match) > 1):
#           new_lbl = Cneuron_cl[match[1]]  
#         else:
#           new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = Cneuron_cl[match[0]]  
    
      
         

    
   
#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       for indx in range(0,len(match)):
#          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#       print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
    

# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])
          

# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


# x = copy.deepcopy(test_X_pois)
# cnt = 0
# correct = 0

# labels = (model.predict(x)).argmax(axis=1) 
# ideals = testFULL_Y




# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]
    
#    # inp = image 
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []

   

#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image 
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []
    
#     inp = image 
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image 
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []

    
#     max_lab7 = False
#     match = -1
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig,support=corr_invs_sel_supp,supp_thres = 50)
#     #print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       labs = []
#       votes = []
#       new_lbl = -1
#       for indx in range(0,len(match)):
#         l = Cneuron_cl[match[indx]]
#         v = 0
#         for indx1 in range(0,len(labs)):
#           if (labs[indx1] == l):
#             votes[indx1] += 1
#             break
#         if (v == 0):
#           labs.append(l)
#           votes.append(v+1)

#       max = -1
#       max_indx = -1
#       for indx in range(0,len(votes)):
#         if (votes[indx] > max):
#           max = votes[indx]
#           max_indx = indx
    
#       if (max_indx == -1):
#         new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = labs[max_indx]

   
#     # match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
#     # print("MATCH:", match)
#     # if (len(match) == 0):
#     #   cnt_not_fnd = cnt_not_fnd + 1
#     #   new_lbl = labels[inp_cnt]
#     # else:
#     #   if (Cneuron_lay[match[0]] == 3):
#     #     max_lab7 = True
#     #     if (len(match) > 1):
#     #       new_lbl = Cneuron_cl[match[1]]  
#     #     else:
#     #       new_lbl = labels[inp_cnt]
#     #   else:
#     #     new_lbl = Cneuron_cl[match[0]]  
#     #   SAME = False
#     #   for indx in range(0,len(match)):
#     #      print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#     #      if ()

    
   
#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       for indx in range(0,len(match)):
#           print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#       print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
    

# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )


In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])
          

# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])



# gen_data = test_X_pois
# gen_labels = test_Y
# actuals_gen = (model.predict(gen_data)).argmax(axis=1)

# x = copy.deepcopy(gen_data)
# cnt = 0
# correct = 0

# labels = actuals_gen #actuals_gen
# ideals = gen_labels #gen_labels




# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]
    
#    # inp = image 
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []

   

#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image 
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []
    
#     inp = image 
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image 
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []

    
#     max_lab7 = False
#     match = -1
#     Same = True
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
#     print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       if (Cneuron_lay[match[0]] == 3):
#         max_lab7 = True
#         if (len(match) > 1):
#           new_lbl = Cneuron_cl[match[1]]  
#         else:
#           new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = Cneuron_cl[match[0]]  
        
#       prev_cls = -1
#       for indx in range(0,len(match)):
#          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#          if (Cneuron_cl[match[indx]] != prev_cls and (Same == True)):
#            Same = False
#            prev_cls = Cneuron_cl[match[indx]]



    
   
#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#   #      print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       if (Same == False):
#         print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
    

# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )
